# Calculate Spatial and temporal NME scores, then calculate ensemble weights.

In [1]:
import os
import math
from copy import deepcopy

import numpy as np
import pandas as pd
import seaborn as sns
import warnings
from tqdm import tqdm

from Functions import *
from Global_Variables import *

In [2]:
obs_df = add_global(pd.read_pickle(os.path.join(RESULTS_PATH, 'AR6_obs_df.pkl')))
obsclim_df = add_global(pd.read_pickle(os.path.join(RESULTS_PATH, 'AR6_obsclim_df.pkl')))
counterclim_df = add_global(pd.read_pickle(os.path.join(RESULTS_PATH, 'AR6_counterclim_df.pkl')))

## Calculate Error scores

### Spatial NME

In [ ]:
warnings.filterwarnings('ignore') # Surpress some unimportant warnings
rng = np.random.default_rng(SEED)
start_year, end_year = 2003, 2019
NME_scores = pd.DataFrame(index=pd.MultiIndex.from_product([obs_dict.keys(), ('$NME_1$', '$NME_3$')], names=['Observation', 'NME']), columns=obsclim_df.columns.unique(level='Model'))

for obsname in NME_scores.index.unique(level='Observation'):
    obs = constrain_time(iris.load_cube(obs_dict[obsname]), start_year, end_year)
    for modelname in NME_scores.columns.unique(level='Model'):
        model = constrain_time(preprocess_model(iris.load_cube(model_dict[modelname]['obsclim']), modelname), start_year, end_year)
        NME_scores.loc[(obsname), modelname] = [NME1(obs, model), NME3(obs, model)]

warnings.filterwarnings('default')   
NME_scores.to_pickle(os.path.join(RESULTS_PATH, 'NME_scores_spatial.pkl'))       

### RMSE

In [3]:
rng = np.random.default_rng(SEED)
start_year, end_year = 2003, 2019
obs_double_df = constrain_time(obs_df, start_year, end_year).stack(sort=False).swaplevel(0, 1).sort_index()
obs_yearly_double_df = to_annual(constrain_time(obs_df, start_year, end_year)).stack(sort=False).swaplevel(0, 1).sort_index()

obsclim_double_df = pd.concat([constrain_time(obsclim_df, start_year, end_year)]*2)
obsclim_yearly_double_df = pd.concat([to_annual(constrain_time(obsclim_df, start_year, end_year))]*2)

RMSE_RA_monthly_df = pd.DataFrame(index=['RMSE'], columns=obsclim_df.columns)
RMSE_RA_annual_df = pd.DataFrame(index=['RMSE'], columns=obsclim_df.columns)

for regionname, modelname in RMSE_RA_monthly_df.columns:
    obs_RA = obs_double_df.loc[slice(None), regionname].groupby('Observation', observed=True).apply(lambda df: relative_anomaly(df)).droplevel(1).droplevel('Observation')
    model_RA = obsclim_double_df.loc[slice(None), regionname].stack('Model').groupby('Model', observed=True).apply(lambda df: relative_anomaly(df)).droplevel(2)
    RMSE_RA_monthly_df.loc['RMSE', (regionname)] = model_RA.groupby('Model', observed=False).apply(lambda model: ((obs_RA - model.droplevel('Model'))**2).mean()**.5).values
    
    obs_yearly_RA = obs_yearly_double_df.loc[slice(None), regionname].groupby('Observation', observed=True).apply(lambda df: relative_anomaly(df)).droplevel(1).droplevel('Observation')
    model_yearly_RA = obsclim_yearly_double_df.loc[slice(None), regionname].stack('Model').groupby('Model', observed=True).apply(lambda df: relative_anomaly(df)).droplevel(2)
    RMSE_RA_annual_df.loc['RMSE', (regionname)] = model_yearly_RA.groupby('Model', observed=False).apply(lambda model: ((obs_yearly_RA - model.droplevel('Model'))**2).mean()**.5).values
    
RMSE_RA_monthly_df.to_pickle(os.path.join(RESULTS_PATH, 'RMSE_RA_monthly.pkl'))
RMSE_RA_annual_df.to_pickle(os.path.join(RESULTS_PATH, 'RMSE_RA_annual.pkl'))

## Optimal sigmaD (1000 times)

### Temporal NME

Calculate __for__ each region, __for__ each of the the two observations the temporal NME3 scores (annual and ranked) of the _fire models_ and the _reference model_ (randomly resampled observations).

In [12]:
rng = np.random.default_rng(SEED)
start_year, end_year = 2003, 2019
index = pd.MultiIndex.from_product([obs_dict.keys(), ('NME3_ranked', 'NME3_annual')], names=['Observation', 'NME'])
NME_scores = pd.DataFrame(index=index, columns=obsclim_df.columns, dtype=np.float32)

columns = obsclim_df.columns.unique(level='Region')
NME_ref_scores = pd.DataFrame(index=index, columns=columns, dtype=np.float32)

for regionname, modelname in NME_scores.columns:
    obs = constrain_time(select_region(obs_df, regionname), start_year, end_year)
    model = constrain_time(select_region(select_models(obsclim_df, modelname), regionname), start_year, end_year)
       
    for obs_name in NME_scores.index.unique(level='Observation'):
        obs_select = select_models(obs, obs_name)
        obs_series, model_series = obs_select[(regionname, obs_name)], model[(regionname, modelname)]
        obs_annual_series, model_annual_series = to_annual(obs_select)[(regionname, obs_name)], to_annual(model)[(regionname, modelname)]
        
        random_obs = pd.Series(rng.choice(obs_select.values.flatten(), size=len(obs.index), replace=False), index=obs_select.index)
        
        NME_scores.loc[(obs_name), (regionname, modelname)] = (NME3_temporal(obs_series.sort_values(), model_series.sort_values()), NME3_temporal(obs_annual_series, model_annual_series))
        NME_ref_scores.loc[(obs_name), (regionname)] = [NME3_temporal(obs_series.sort_values(), random_obs), NME3_temporal(to_annual(obs_series), to_annual(random_obs))]

NME_scores.to_pickle(os.path.join(RESULTS_PATH, 'NME_scores_temporal.pkl'))
NME_ref_scores.to_pickle(os.path.join(RESULTS_PATH, 'NME_ref_scores_temporal.pkl'))

In [5]:
obs_double_df = constrain_time(obs_df, start_year, end_year).stack(sort=False).swaplevel(0, 1).sort_index()
obsclim_double_df = pd.concat([constrain_time(obsclim_df, start_year, end_year)]*2)
RMSE_RA_monthly_df = pd.DataFrame(index=obsclim_df.columns.unique(level='Region'), columns=obsclim_df.columns.unique(level='Model'))

for regionname, modelname in obsclim_double_df.columns:
    obs_series = obs_double_df.loc[slice(None), regionname]
    model_series = obsclim_double_df.loc[slice(None), (regionname, modelname)]
    obs_RA = (obs_series-obs_series.mean())/obs_series.mean()
    model_RA = (model_series-model_series.mean())/model_series.mean()
    RMSE_RA_monthly_df.loc[regionname, modelname] = ((obs_RA.values - model_RA.values)**2).mean()**.5
    
RMSE_RA_monthly_df = RMSE_RA_monthly_df.astype(np.float32)
RMSE_RA_monthly_df.to_pickle(os.path.join(RESULTS_PATH, 'RMSE_RA.pkl'))

In [6]:
max_sigmaD = 10
sigmaDs = pd.Series(np.logspace(0.01, np.log2(max_sigmaD+1), base=2,num=500, dtype=np.float32)-1, name='sigmaD')
model_weights = pd.DataFrame(np.nan, index=sigmaDs, columns=obsclim_df.columns, dtype=np.float64)

distances = pd.to_numeric((NME_scores/NME_scores.T.groupby(level='Region', sort=False, observed=True).median().T).sum())
for sigmaD in model_weights.index:
    weights_not_normalized = (-distances/sigmaD).apply(lambda x: np.exp(x))
    weights_normalized = weights_not_normalized/weights_not_normalized.groupby(level=('Region'), sort=False, observed=True).sum()
    model_weights.loc[(sigmaD), slice(None)] = weights_normalized

In [7]:
obsclim_pd = constrain_time(obsclim_df, start_year, end_year)
obsclim_pd_RA = relative_anomaly(pd.concat([obsclim_pd]*2))
obs_pd_RA_df = relative_anomaly(constrain_time(obs_df, start_year, end_year)).stack(sort=False).swaplevel(0, 1).sort_index().droplevel('Observation')

num_loops = 1000
fraction_correct = []

for idx in tqdm(range(num_loops)):
    obsclim_pd_RA_added_noise = add_error(obsclim_pd_RA, seed=idx, error=RMSE_RA_monthly_df.stack())
    fraction_correct_idx = []
    
    for sigmaD in sigmaDs:
        weights = model_weights.loc[sigmaD]
        weighted_average = (obsclim_pd_RA_added_noise*weights).T.groupby(level='Region', sort=False, observed=True).sum().T
        
        variance = obsclim_pd_RA_added_noise.subtract(weighted_average, axis=0)**2
        weighted_variance = variance*weights
        weighted_variance_regional = weighted_variance.T.groupby(level='Region', sort=False, observed=True).sum().T
        weighted_stddev = np.sqrt(weighted_variance_regional)
        
        upper_bound = weighted_average+1.96*weighted_stddev
        lower_bound = weighted_average-1.96*weighted_stddev
        
        fraction_correct_idx_sigmaD = ((obs_pd_RA_df <= upper_bound) & (obs_pd_RA_df >= lower_bound)).mean(axis=0)
        fraction_correct_idx_sigmaD.name = sigmaD
        fraction_correct_idx.append(fraction_correct_idx_sigmaD)
        
    fraction_correct_idx = pd.concat(fraction_correct_idx, axis=1).T
    fraction_correct_idx.index.name = 'sigmaD'
    fraction_correct.append(fraction_correct_idx)
    
fraction_correct = pd.concat(fraction_correct, keys=np.arange(num_loops), names=['idx', 'sigmaD'])
fraction_correct.to_pickle(os.path.join(RESULTS_PATH, 'fraction_correct.pkl'))

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [59:34<00:00,  3.57s/it]


In [9]:
# Transform the fraction of correct values to a boolean indicating whetehr it's above 0.95 (True) or not (False).
# Then groupby idx and region.
fraction_correct_over_95 = fraction_correct>0.95
fraction_correct_over_95_grouped = fraction_correct_over_95.stack().unstack('sigmaD').groupby(['idx', 'Region'], sort=False, observed=False)
# For each idx, region combination take the idxmax (first idx (=sigmaD) at which we find the max value (True)), if none of the values are True then we take np.nan instead
optimal_sigmaDs = fraction_correct_over_95_grouped.apply(lambda x: x.idxmax(axis=1).values.item() if x.any(axis=1).values.item() else np.nan)
optimal_sigmaDs = optimal_sigmaDs.unstack('Region')

optimal_weights = pd.DataFrame(index=optimal_sigmaDs.index, columns=model_weights.columns, dtype=np.float32)
for regionname in optimal_sigmaDs.columns:
    region_weights = model_weights.loc[(optimal_sigmaDs.loc[slice(None), regionname].dropna()), (regionname)]
    optimal_weights.loc[(~optimal_sigmaDs.loc[slice(None), regionname].isna()), (regionname)] = region_weights.values.astype(np.float32)

optimal_weights.index = optimal_weights.index.astype(np.int16)
optimal_weights.to_pickle(os.path.join(RESULTS_PATH, 'optimal_weights.pkl'))